### Importar bibliotecas

In [1]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

##### Ler os dados das ações

In [2]:
dados = pd.read_csv('../dados/historico_acoes.csv', encoding='latin-1')

dados.head()

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TIPO,DATA,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
0,1,19880104,2.0,ABC 6,10,ABC XTAL,PPA,NaN,CZ$,225.0,...,2.0,3000.0,685000.0,0.0,0.0,99991231.0,1.0,0.0,ABCXACPPA,101.0
1,1,19880104,2.0,ACE 2,10,ACESITA,PP C01,NaN,CZ$,440.0,...,1.0,400.0,176000.0,0.0,0.0,99991231.0,1.0,0.0,ACESACPP,114.0
2,1,19880104,2.0,EAL 2,10,ACO ALTONA,PP,NaN,CZ$,863.0,...,8.0,33100.0,29863800.0,0.0,0.0,99991231.0,1.0,0.0,EALTACPP,117.0
3,1,19880104,2.0,AVI 2,10,ACOS VILL,PP C42,NaN,CZ$,330.0,...,12.0,112700.0,37149000.0,0.0,0.0,99991231.0,1.0,0.0,AVILACPP,140.0
4,1,19880104,2.0,CRA 2,10,ADUBOS CRA,PP C31,NaN,CZ$,90.0,...,1.0,10000.0,900000.0,0.0,0.0,99991231.0,1.0,0.0,CRADACPP,121.0


##### Quantidade de ações presentes nos dados

In [3]:
len(dados.CODNEG.unique())

141139

##### Filtrando apenas as colunas úteis

In [4]:
dados = dados[['DATA', 'CODNEG', 'PREABE', 'PREULT']]

dados.head()

,DATA,CODNEG,PREABE,PREULT
0,19880104,ABC 6,225.0,230.0
1,19880104,ACE 2,440.0,440.0
2,19880104,EAL 2,863.0,920.0
3,19880104,AVI 2,330.0,330.0
4,19880104,CRA 2,90.0,90.0


### Criar variável classe

A variável classe representa se a ação teve desempenho positivo ou negativo no dia.

Desempenho positivo: O valor final da ação no dia fechou maior ou igual ao valor de abertura

Desempenho negativo: Se o valor de fechamento for menor que o valor de abertura, entao a ação teve diminuição/queda, logo desempenho negativo

In [5]:
# Criar variável classe
dados['STATUS_POSITIVO'] = 1

# Se o valor de fechamento for menor que o valor de abertura, entao a ação teve diminuição/queda
dados.loc[dados['PREULT']<dados['PREABE'],'STATUS_POSITIVO'] = 0

In [6]:
dados['STATUS_POSITIVO'].unique()

array([1, 0])

##### Verificando os dados das ações PETR4 e VALE3

In [7]:
dados.loc[dados['CODNEG']=='PETR4'].head()

,DATA,CODNEG,PREABE,PREULT,STATUS_POSITIVO
168517,20080102,PETR4,8699.0,8700.0,1
169690,20080103,PETR4,8670.0,8560.0,0
170909,20080104,PETR4,8565.0,8140.0,0
172103,20080107,PETR4,8146.0,7901.0,0
173318,20080108,PETR4,8030.0,8335.0,1


In [8]:
dados.loc[dados['CODNEG']=='VALE3'].head()

,DATA,CODNEG,PREABE,PREULT,STATUS_POSITIVO
168896,20080102,VALE3,5925.0,5790.0,0
170049,20080103,VALE3,5806.0,5820.0,1
171307,20080104,VALE3,5870.0,5610.0,0
172460,20080107,VALE3,5648.0,5510.0,0
173717,20080108,VALE3,5600.0,5620.0,1


***
### Lendo dimensão data

In [9]:
dimensao_data = pd.read_csv('../dados/dimensao_data.csv', sep=';')

dimensao_data.head()

,data,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,bimestre,semestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
0,19850101,1985,1,1,1,3,1,1,1,1,...,1,1,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
1,19850102,1985,1,2,2,4,1,1,1,1,...,1,2,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
2,19850103,1985,1,3,3,5,1,1,1,1,...,1,3,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
3,19850104,1985,1,4,4,6,1,1,1,1,...,1,4,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
4,19850105,1985,1,5,5,7,1,1,1,1,...,1,5,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31


In [10]:
dimensao_data.tail()

,data,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,bimestre,semestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
29215,20641227,2064,12,362,27,7,52,4,6,4,...,4,88,2064-12-22,2064-12-28,2064-12-01,2064-12-31,2064-10-01,2064-12-31,2064-01-01,2064-12-31
29216,20641228,2064,12,363,28,1,1,4,6,4,...,4,89,2064-12-22,2064-12-28,2064-12-01,2064-12-31,2064-10-01,2064-12-31,2064-01-01,2064-12-31
29217,20641229,2064,12,364,29,2,1,4,6,4,...,5,90,2064-12-29,2065-01-04,2064-12-01,2064-12-31,2064-10-01,2064-12-31,2065-01-01,2065-12-31
29218,20641230,2064,12,365,30,3,1,4,6,4,...,5,91,2064-12-29,2065-01-04,2064-12-01,2064-12-31,2064-10-01,2064-12-31,2065-01-01,2065-12-31
29219,20641231,2064,12,366,31,4,1,4,6,4,...,5,92,2064-12-29,2065-01-04,2064-12-01,2064-12-31,2064-10-01,2064-12-31,2065-01-01,2065-12-31


##### Renomeando coluna "key" para unir com os dados das ações

In [11]:
dimensao_data=dimensao_data.rename(columns = {'data':'DATA'})

dimensao_data.head()

,DATA,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,bimestre,semestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
0,19850101,1985,1,1,1,3,1,1,1,1,...,1,1,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
1,19850102,1985,1,2,2,4,1,1,1,1,...,1,2,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
2,19850103,1985,1,3,3,5,1,1,1,1,...,1,3,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
3,19850104,1985,1,4,4,6,1,1,1,1,...,1,4,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31
4,19850105,1985,1,5,5,7,1,1,1,1,...,1,5,1984-12-31,1985-01-06,1985-01-01,1985-01-31,1985-01-01,1985-03-31,1985-01-01,1985-12-31


In [12]:
dimensao_data.dtypes

DATA                          int64
ano                           int64
mes                           int64
dia_do_ano                    int64
dia_do_mes                    int64
dia_da_semana                 int64
semana_do_ano                 int64
trimestre                     int64
bimestre                      int64
semestre                      int64
quinzena                      int64
semana_do_mes                 int64
dia_do_trimestre              int64
primeiro_dia_da_semana       object
ultimo_dia_da_semana         object
primeiro_dia_do_mes          object
ultimo_dia_do_mes            object
primeiro_dia_do_trimestre    object
ultimo_dia_do_trimestre      object
primeiro_dia_do_ano          object
ultimo_dia_do_ano            object
dtype: object

##### Convertendo as datas que estão no formato YYYY-MM-DD para YYYYMMDD, 
##### Essa conversão é necessária para esses atributos ficarem do tipo int, pois com o "-" o atributo fica do tipo object, o que prejudica o desempenho das técnicas

In [13]:
dimensao_data['primeiro_dia_da_semana'] = dimensao_data['primeiro_dia_da_semana'].str.replace('-','').apply(int)
dimensao_data['ultimo_dia_da_semana'] = dimensao_data['ultimo_dia_da_semana'].str.replace('-','').apply(int)
dimensao_data['primeiro_dia_do_mes'] = dimensao_data['primeiro_dia_do_mes'].str.replace('-','').apply(int)
dimensao_data['ultimo_dia_do_mes'] = dimensao_data['ultimo_dia_do_mes'].str.replace('-','').apply(int)
dimensao_data['primeiro_dia_do_trimestre'] = dimensao_data['primeiro_dia_do_trimestre'].str.replace('-','').apply(int)
dimensao_data['ultimo_dia_do_trimestre'] = dimensao_data['ultimo_dia_do_trimestre'].str.replace('-','').apply(int)
dimensao_data['primeiro_dia_do_ano'] = dimensao_data['primeiro_dia_do_ano'].str.replace('-','').apply(int)
dimensao_data['ultimo_dia_do_ano'] = dimensao_data['ultimo_dia_do_ano'].str.replace('-','').apply(int)

dimensao_data.dtypes

DATA                         int64
ano                          int64
mes                          int64
dia_do_ano                   int64
dia_do_mes                   int64
dia_da_semana                int64
semana_do_ano                int64
trimestre                    int64
bimestre                     int64
semestre                     int64
quinzena                     int64
semana_do_mes                int64
dia_do_trimestre             int64
primeiro_dia_da_semana       int64
ultimo_dia_da_semana         int64
primeiro_dia_do_mes          int64
ultimo_dia_do_mes            int64
primeiro_dia_do_trimestre    int64
ultimo_dia_do_trimestre      int64
primeiro_dia_do_ano          int64
ultimo_dia_do_ano            int64
dtype: object

In [14]:
dimensao_data.head()

,DATA,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,bimestre,semestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
0,19850101,1985,1,1,1,3,1,1,1,1,...,1,1,19841231,19850106,19850101,19850131,19850101,19850331,19850101,19851231
1,19850102,1985,1,2,2,4,1,1,1,1,...,1,2,19841231,19850106,19850101,19850131,19850101,19850331,19850101,19851231
2,19850103,1985,1,3,3,5,1,1,1,1,...,1,3,19841231,19850106,19850101,19850131,19850101,19850331,19850101,19851231
3,19850104,1985,1,4,4,6,1,1,1,1,...,1,4,19841231,19850106,19850101,19850131,19850101,19850331,19850101,19851231
4,19850105,1985,1,5,5,7,1,1,1,1,...,1,5,19841231,19850106,19850101,19850131,19850101,19850331,19850101,19851231


##### Salvando dimensão data tratada

In [15]:
dimensao_data.to_csv('../dados/dimensao_data_tratado.csv', sep=';', index=False)

##### Selecionando dados das ações para realizar a união com a dimensão data

In [16]:
dados_selecionados = dados[['DATA', 'CODNEG', 'STATUS_POSITIVO']]

dados_selecionados.head()

,DATA,CODNEG,STATUS_POSITIVO
0,19880104,ABC 6,1
1,19880104,ACE 2,1
2,19880104,EAL 2,1
3,19880104,AVI 2,1
4,19880104,CRA 2,1


***
### Unindo dimensão tempo com os dados das ações

In [17]:
completo = pd.merge(dados_selecionados, dimensao_data, on='DATA')

completo.head()

,DATA,CODNEG,STATUS_POSITIVO,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
0,19880104,ABC 6,1,1988,1,4,4,2,2,1,...,1,4,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
1,19880104,ACE 2,1,1988,1,4,4,2,2,1,...,1,4,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
2,19880104,EAL 2,1,1988,1,4,4,2,2,1,...,1,4,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
3,19880104,AVI 2,1,1988,1,4,4,2,2,1,...,1,4,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
4,19880104,CRA 2,1,1988,1,4,4,2,2,1,...,1,4,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231


In [18]:
completo.dtypes

DATA                          int64
CODNEG                       object
STATUS_POSITIVO               int64
ano                           int64
mes                           int64
dia_do_ano                    int64
dia_do_mes                    int64
dia_da_semana                 int64
semana_do_ano                 int64
trimestre                     int64
bimestre                      int64
semestre                      int64
quinzena                      int64
semana_do_mes                 int64
dia_do_trimestre              int64
primeiro_dia_da_semana        int64
ultimo_dia_da_semana          int64
primeiro_dia_do_mes           int64
ultimo_dia_do_mes             int64
primeiro_dia_do_trimestre     int64
ultimo_dia_do_trimestre       int64
primeiro_dia_do_ano           int64
ultimo_dia_do_ano             int64
dtype: object

##### Quantidade de tuplas:

In [19]:
len(completo)

7237112

##### Verificando se existe algum valor nulo nos dados

In [20]:
completo.isnull().values.any()

False

##### Salvando os dados

In [21]:
completo.to_csv('../dados/dados_tratados.csv', index=False)

***
### Filtro: Manter os dados referentes apenas das ações que estão no mercado atualmente

Ações que já encerraram sua participação no mercado de ações não nós interessa.

##### Verificando qual último dia presente nos dados

In [22]:
ultimo_dia = completo.loc[completo['DATA'].astype(str).str[:4] == '2018', 'DATA'].unique()
ultimo_dia = ultimo_dia[-1:]
ultimo_dia

array([20181024])

##### Verificando quais ações operaram no último dia

In [23]:
acoes_no_ultimo_dia = completo.loc[completo['DATA'] == int(ultimo_dia), 'CODNEG'].unique()
acoes_no_ultimo_dia

array(['AALR3', 'AALR3F', 'AAPL34', ..., 'XPLG11', 'XPML11', 'XTED11'], dtype=object)

##### Quantidade de ações no último dia

In [24]:
len(acoes_no_ultimo_dia)

2265

##### Selecionando os dados apenas das ações atuais

In [25]:
dados_atuais = completo.loc[completo['CODNEG'].isin(acoes_no_ultimo_dia)]
dados_atuais.head()

,DATA,CODNEG,STATUS_POSITIVO,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
167674,20080102,ABCB4,0,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167675,20080102,ABCB4F,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167701,20080102,AGRO3,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167714,20080102,ALPA4,0,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167715,20080102,ALPA4F,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231


#### Removendo ações que existe somente 1 valor para o atributo classe ou que tenham menos de 30 tuplas

- Remover ações que o atributo 'STATUS_POSITIVO' só tem 1 valor, somente positivo (1), ou somente negativo (0)

- Remover ações que tenham menos do que 30 tuplas de dados

In [26]:
# Coletando as ações
acoes = dados_atuais.CODNEG.unique()

##### Função para verificar se a ação será removida ou não, com base no que foi definido acima

In [27]:
def verificarAcao(acao):
    classes = dados_atuais.loc[dados_atuais['CODNEG'] == acao, 'STATUS_POSITIVO'].unique()
    tuplas = dados_atuais.loc[dados_atuais['CODNEG'] == acao]
    if (len(classes) == 1) or (len(tuplas) < 30): # Se a ação atender a algum desses critérios ela será removida
        print(acao)
        return acao        

##### Realizar a verificação de cada ação, de forma paralela, executando em todos os núcleos do processador

In [28]:
remover_acoes = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(verificarAcao)(acao) for acao in acoes)

MTSA4T
ELETS25
ELETG25
ITSAX10
PETRW29
CSNAK2
VALEM55
VALEW22
AZEV3F
IBOVB90
ITSAX2
PETRM53
LRENM35
AZEV4T
BBASK83
BBDCX83
BOVAL87
LRENA38
BBDCM332
LRENN35
VALEL506
BBASL402
IBOVW79
BBDCL394
PETRX172
BBDCX351
PETRM170
BRMLM41
CEED4F
BBDCX366
ITUBB515
VALEL518
LRENN32
PETRL215
ITUBL531
BBDCX361
BBASL417
VALEL543
BBDCX334
BBDCX339
PETRX182
BBASX304
BBDCX341
BBDCX376
ITUBN456
BBDCA425
BBASX394
BBDCX411
BBASL469
IBOVX79
BBASX387
BBASL459
BBDCL471
CYREW14
PETRA275
BBASK419
BBDCX379
GGBRK159
VALEL548
NPAR11
PETRW177
IBOVL102
PETRW197
BBASX347
BBASL439
BBASL407
PETRL205
PETRL350
BRMLK72
IBOVX85
LRENW342
VALEX395
NATUX33
PETRP220
GOAUK830
VALEL462
BBASX374
BBDCD167
BBDCD201
ABEVE227
BBASX376
ABEVQ195
ITUBK244
ITUBK234
BRAPL34
BBASL399
BBASL489
ITUBK574
BRFSW21
VALEL549
BBDCK369
PETRL265
BBASW364
BBASW324
VALEL522
BBASK422
ITSAK145
ITUBW462
ITSAK135
PETRM210
ITUBW504
BBASL374
BBASL424
RAILX14
PETRM232
BBASK427
BBASW367
BBASK377
BBASL44
BBASX371
BRMLK62
BRMLW11
ABEVK43
BRMLL12
PETRK257
ITUBL490


##### Removendo valores nulos da lista de ações a serem removidas

In [29]:
remover_acoes = list(filter(None, remover_acoes))

##### Quantidade de ações a serem removidas

In [30]:
len(remover_acoes)

985

##### Removendo ações

In [31]:
dados_atuais = dados_atuais.loc[~dados_atuais['CODNEG'].isin(remover_acoes)]

dados_atuais.head()

,DATA,CODNEG,STATUS_POSITIVO,ano,mes,dia_do_ano,dia_do_mes,dia_da_semana,semana_do_ano,trimestre,...,semana_do_mes,dia_do_trimestre,primeiro_dia_da_semana,ultimo_dia_da_semana,primeiro_dia_do_mes,ultimo_dia_do_mes,primeiro_dia_do_trimestre,ultimo_dia_do_trimestre,primeiro_dia_do_ano,ultimo_dia_do_ano
167674,20080102,ABCB4,0,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167675,20080102,ABCB4F,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167701,20080102,AGRO3,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167714,20080102,ALPA4,0,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167715,20080102,ALPA4F,1,2008,1,2,2,4,1,1,...,1,2,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231


##### Total de ações restante no dataset

In [32]:
len(dados_atuais.CODNEG.unique())

1280

In [36]:
pd.DataFrame(dados_atuais.CODNEG.unique(), columns=['CODNEG']).to_csv('../dados/acoes_verificadas.csv', sep=';', index=False)

##### Verificando quantidade de tuplas

In [37]:
len(dados_atuais)

1985466

#### O conjunto de dados diminuiu de **7,2 milhões** de tuplas para **1 milhão e 900 mil** tuplas aproximadamente

***
Salvando os dados tratados

In [38]:
dados_atuais.to_csv('../dados/dados_tratados_acoes_atuais.csv', index=False)